In [1]:
import pandas as pd
import inputs 
from core.position import position_holding_returns, position_historical_returns
from core.statistics import *
from core.performance import *
from core.logic import Momentum

In [2]:
position_df = pd.read_csv(inputs.POSITION_PATH, index_col = 0)

historical_returns_df = position_historical_returns(inputs.MARKET,position_df.index.values.tolist(),position_df['actual_weights'].tolist(),period=inputs.PERIOD)

holding_returns_df = position_holding_returns(inputs.MARKET,position_df.index.values.tolist(),position_df['actual_weights'].tolist(),inputs.POSITION_PURCHASE_DATE)

holding_cumulative_returns_df = cumulative_returns(holding_returns_df)

holding_cumulative_returns_s = holding_cumulative_returns_df.iloc[-1] -1

historical_returns_df

[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  1 of 1 completed


,position,^RUA
Date,,
2021-03-11,0.047909,0.012878
2021-03-12,0.012705,0.001374
2021-03-15,0.008514,0.007085
2021-03-16,0.000361,-0.003661
2021-03-17,-0.021587,0.003177
...,...,...
2023-03-03,0.051749,0.016014
2023-03-06,0.010414,-0.001377
2023-03-07,0.006074,-0.014719


In [3]:
pop_stats_df = population_stats(historical_returns_df)
sample_stats_df  = sample_stats(holding_returns_df,inputs.POSITION_PURCHASE_DATE)

hypothesis_df = hypothesis_test(sample_stats_df,pop_stats_df)

hypothesis_df


,samplereject
position,False
^RUA,False


In [4]:
comparison_dct = regression_comparison(historical_returns_df,holding_returns_df,inputs.MARKET)

comparison_dct['comparison_df']

tickers,historical_timeframe,holding_timeframe,before_purchase
alpha,0.001948,0.006051,0.001547
betav,0.800741,1.258417,0.766574
alpha_t_stat,2.16387,1.429596,1.728061
beta_t_stat,11.170888,3.197274,10.92389
skewness,0.564,1.336,0.230
kurtoses,8.082,10.576,6.247
betaub,0.941573,2.052168,0.904479
betalb,0.659908,0.464666,0.628669
alphalb,0.000179,-0.002485,-0.000212
alphaub,0.003717,0.014587,0.003306


In [5]:
is_beta_rebalance_tactical = Momentum.smart_beta_signal(comparison_dct['comparison_df'],comparison_dct['recent_period_return'])

is_beta_rebalance_tactical

True

In [6]:
is_alpha_rebalance_tactical = Momentum.smart_alpha_signal(comparison_dct['comparison_df'],comparison_dct['recent_period_return'])

is_alpha_rebalance_tactical

False

In [7]:
pred_comparison_dct = prediction_comparison(historical_returns_df, holding_returns_df,inputs.MARKET)

is_prediction_rebalance_tactical = Momentum.prediction_signal(pred_comparison_dct['comparison_df'],pred_comparison_dct['recent_period_return'])

is_prediction_rebalance_tactical

False

In [8]:
timing = TM_timing(holding_returns_df,inputs.MARKET)

timing

-4.14918642944169